In [10]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
import os
from flask_marshmallow import Marshmallow

app4 = Flask(__name__)
basedir = os.path.abspath(os.path.dirname('__file__'))
app4.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir,'product')
app4.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False 

db = SQLAlchemy(app4)
ma = Marshmallow(app4)

class Product(db.Model):
    id = db.Column(db.Integer , primary_key=True)
    name = db.Column(db.String(50), unique=True)
    description = db.Column(db.String(200), nullable = False)
    price = db.Column(db.Float)
    qty = db.Column(db.Integer)
    
    def __init__(self, name, description , price , qty):
        self.name = name
        self.description = description
        self.price = price
        self.qty = qty
        
class ProductSchema(ma.Schema):
    class Meta:
        fields = ('id', 'name', 'description' , 'price', 'qty')
        
Product_Schema = ProductSchema()
Products_Schema = ProductSchema(many = True)        

#Inserting a product
@app4.route("/product", methods= ['POST'])
def add_product():
    name = request.json['name']
    description = request.json['description']
    price = request.json['price']
    qty = request.json['qty']

    new_product = Product(name , description, price, qty)

    db.session.add(new_product)
    db.session.commit()
    
    return Product_Schema.jsonify(new_product)

#Getting all the products
@app4.route('/product', methods= ['GET'])
def get_products():
    all_products = Product.query.all()
    result = Products_Schema.dump(all_products)
    return jsonify(result)

#Gettning a single product
@app4.route('/product/<id>', methods= ['GET'])
def get_product(id):
    product = Product.query.get(id)
    result = Product_Schema.dump(product)
    return jsonify(result)

#Updating a product
@app4.route("/product/<id>", methods= ['PUT'])
def update(id):
    product = Product.query.get(id)
                                       
    name = request.json['name']
    description = request.json['description']
    price = request.json['price']
    qty = request.json['qty']
    
    product.name = name
    product.description = description
    product.price = price
    product.qty = qty
                                       
    db.session.commit()
    
    return Product_Schema.jsonify(product)

#Deleteing a product
@app4.route('/product/<id>', methods= ['DELETE'])
def delete_product(id):
    product = Product.query.get(id)
    db.session.delete(product)
    db.session.commit()
    
    return Product_Schema.jsonify(product)

app4.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Nov/2021 14:20:10] "GET /product HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 14:20:16] "DELETE /product/5 HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 14:20:21] "GET /product HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 14:20:37] "DELETE /product/2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 14:20:40] "GET /product HTTP/1.1" 200 -
